In [1]:
import pandas as pd
import numpy as np
import folium
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler

# Laad de gebruikersprofielen en routes
user_profiles_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'
routes_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'
user_profiles_df = pd.read_json(user_profiles_path, lines=True)
routes_df = pd.read_json(routes_path, lines=True)

# Preprocessing en normalisatie
scaler = StandardScaler()
routes_df['mean_altitude'] = routes_df['altitude'].apply(np.mean)
user_profiles_df[['most_common_lat', 'most_common_lon']] = pd.DataFrame(user_profiles_df['average_location'].tolist(), index=user_profiles_df.index)
user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']] = scaler.fit_transform(user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']])
routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']] = scaler.fit_transform(routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']])

# Bepaal gewichten voor gebruikerskenmerken
gewichten = {row['userId']: {'average_altitude_change': row['weight_std_altitude_change'], 
                            'average_distance': row['weight_std_distance'], 
                            'average_condition_value': row['weight_std_condition_value'],
                            'distance_to_user': 1.0} for index, row in user_profiles_df.iterrows()}

# Functie om de afstand tussen twee punten te berekenen
def calculate_distance(point1, point2):
    return geodesic(point1, point2).kilometers

# Functie om een aansluitend route segment te vinden
def find_next_segment(current_location, remaining_segments, user_profile_weights):
    closest_segment = None
    min_distance = float('inf')
    
    for index, segment in remaining_segments.iterrows():
        segment_start_location = (segment['latitude'][0], segment['longitude'][0])
        distance_to_segment = calculate_distance(current_location, segment_start_location)

        if distance_to_segment < min_distance:
            min_distance = distance_to_segment
            closest_segment = segment
    
    return closest_segment

# Functie voor het genereren van nieuwe routes
def generate_new_route(user_id, user_profiles_df, routes_df, max_distance=10):
    user_profile = user_profiles_df[user_profiles_df['userId'] == user_id].iloc[0]
    user_location = (user_profile['most_common_lat'], user_profile['most_common_lon'])
    
    new_route_segments = []
    current_location = user_location
    remaining_distance = max_distance

    while remaining_distance > 0:
        remaining_segments = routes_df[routes_df['distance'] <= remaining_distance]
        next_segment = find_next_segment(current_location, remaining_segments, gewichten[user_id])

        if next_segment is not None:
            new_route_segments.append(next_segment)
            current_location = (next_segment['latitude'][-1], next_segment['longitude'][-1])
            remaining_distance -= next_segment['distance']
        else:
            break

    return pd.concat(new_route_segments)

# Test de functie voor een specifieke gebruiker
sample_user_id = 367324
new_route = generate_new_route(sample_user_id, user_profiles_df, routes_df)

# Zoek het gebruikersprofiel voor visualisatie
user_profile = user_profiles_df[user_profiles_df['userId'] == sample_user_id].iloc[0]

# Visualiseer de nieuwe route
map_obj = folium.Map(location=[user_profile['most_common_lat'], user_profile['most_common_lon']], zoom_start=12)
popup_text = f"UserID: {sample_user_id}<br>Average Speed: {user_profile['average_speeds']}<br>Average Altitude Change: {user_profile['average_altitude_changes']}<br>Average Distance: {user_profile['average_distances']}<br>Average Condition Value: {user_profile['average_condition_value']}"
folium.Marker(location=[user_profile['most_common_lat'], user_profile['most_common_lon']], popup=popup_text).add_to(map_obj)

for _, segment in new_route.iterrows():
    segment_coords = list(zip(segment['latitude'], segment['longitude']))
    folium.PolyLine(segment_coords, color='blue', weight=2.5, opacity=1).add_to(map_obj)

map_obj


In [2]:
import pandas as pd
import numpy as np
import folium
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler

# Laad de gebruikersprofielen en routes
user_profiles_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'
routes_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'
user_profiles_df = pd.read_json(user_profiles_path, lines=True)
routes_df = pd.read_json(routes_path, lines=True)

# Preprocessing en normalisatie
scaler = StandardScaler()
routes_df['mean_altitude'] = routes_df['altitude'].apply(np.mean)
user_profiles_df[['most_common_lat', 'most_common_lon']] = pd.DataFrame(user_profiles_df['average_location'].tolist(), index=user_profiles_df.index)
user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']] = scaler.fit_transform(user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']])
routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']] = scaler.fit_transform(routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']])

# Bepaal gewichten voor gebruikerskenmerken
gewichten = {row['userId']: {'average_altitude_change': row['weight_std_altitude_change'], 
                            'average_distance': row['weight_std_distance'], 
                            'average_condition_value': row['weight_std_condition_value'],
                            'distance_to_user': 1.0} for index, row in user_profiles_df.iterrows()}

# Functie om de afstand tussen twee punten te berekenen
def calculate_distance(point1, point2):
    return geodesic(point1, point2).kilometers

# Functie om een aansluitend route segment te vinden
def find_next_segment(current_location, remaining_segments, user_profile_weights):
    closest_segment = None
    min_distance = float('inf')
    
    for index, segment in remaining_segments.iterrows():
        segment_start_location = (segment['latitude'][0], segment['longitude'][0])
        distance_to_segment = calculate_distance(current_location, segment_start_location)

        if distance_to_segment < min_distance:
            min_distance = distance_to_segment
            closest_segment = segment
    
    return closest_segment

def generate_new_route(user_id, user_profiles_df, routes_df, max_distance=10, search_radius=5):
    user_profile = user_profiles_df[user_profiles_df['userId'] == user_id].iloc[0]
    user_location = (user_profile['most_common_lat'], user_profile['most_common_lon'])
    
    # Selecteer segmenten binnen een bepaalde straal van de gebruiker
    nearby_segments = []
    for _, segment in routes_df.iterrows():
        segment_start_location = (segment['latitude'][0], segment['longitude'][0])
        distance_to_user = calculate_distance(user_location, segment_start_location)
        if distance_to_user <= search_radius:
            nearby_segments.append(segment)
    
    # Sorteer de nabijgelegen segmenten op afstand tot de gebruiker
    nearby_segments.sort(key=lambda x: calculate_distance(user_location, (x['latitude'][0], x['longitude'][0])))

    # Bouw de route op uit de nabijgelegen segmenten
    new_route_segments = []
    current_location = user_location
    remaining_distance = max_distance

    for segment in nearby_segments:
        if segment['distance'] <= remaining_distance:
            new_route_segments.append(segment)
            current_location = (segment['latitude'][-1], segment['longitude'][-1])
            remaining_distance -= segment['distance']
            if remaining_distance <= 0:
                break

    # Combineer de segmenten tot een DataFrame
    if new_route_segments:
        return pd.concat(new_route_segments, ignore_index=True)
    else:
        return pd.DataFrame()  # Geen route gevonden, retourneer een lege DataFrame

# Test de functie voor een specifieke gebruiker
sample_user_id = 367324
new_route = generate_new_route(sample_user_id, user_profiles_df, routes_df)


# Zoek het gebruikersprofiel voor visualisatie
user_profile = user_profiles_df[user_profiles_df['userId'] == sample_user_id].iloc[0]

# Visualiseer de nieuwe route
map_obj = folium.Map(location=[user_profile['most_common_lat'], user_profile['most_common_lon']], zoom_start=12)
popup_text = f"UserID: {sample_user_id}<br>Average Speed: {user_profile['average_speeds']}<br>Average Altitude Change: {user_profile['average_altitude_changes']}<br>Average Distance: {user_profile['average_distances']}<br>Average Condition Value: {user_profile['average_condition_value']}"
folium.Marker(location=[user_profile['most_common_lat'], user_profile['most_common_lon']], popup=popup_text).add_to(map_obj)

for _, segment in new_route.iterrows():
    segment_coords = list(zip(segment['latitude'], segment['longitude']))
    folium.PolyLine(segment_coords, color='blue', weight=2.5, opacity=1).add_to(map_obj)

map_obj


AttributeError: 'Series' object has no attribute 'iterrows'

In [11]:
import pandas as pd
import numpy as np
import folium
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

# Laad de gebruikersprofielen en routes
user_profiles_df = pd.read_json(user_profiles_path, lines=True)
routes_df = pd.read_json(routes_path, lines=True)

# Preprocessing en normalisatie
scaler = StandardScaler()
routes_df['mean_altitude'] = routes_df['altitude'].apply(np.mean)
user_profiles_df[['most_common_lat', 'most_common_lon']] = pd.DataFrame(user_profiles_df['locations'].apply(lambda x: np.mean(x, axis=0)).tolist(), index=user_profiles_df.index)
user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']] = scaler.fit_transform(user_profiles_df[['speeds', 'altitude_changes', 'distances', 'average_condition_value']])
routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']] = scaler.fit_transform(routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']])

# Bepaal gewichten voor gebruikerskenmerken
gewichten = user_profiles_df.set_index('userId')[['weight_std_altitude_change', 'weight_std_distance', 'weight_std_condition_value']].to_dict('index')

# Vectorized functie voor afstandsberekening
def calculate_distances(points1, points2):
    return cdist(points1, points2, metric='euclidean')

# Functie om een aansluitend route segment te vinden
def find_next_segment(current_location, remaining_segments):
    segment_start_locations = remaining_segments.apply(lambda x: (x['latitude'][0], x['longitude'][0]), axis=1).tolist()
    distances = calculate_distances([current_location], segment_start_locations)
    min_index = np.argmin(distances)
    return remaining_segments.iloc[min_index]

# Functie voor het genereren van nieuwe routes
def generate_new_route(user_id, max_distance=10):
    user_profile = user_profiles_df[user_profiles_df['userId'] == user_id].iloc[0]
    user_location = (user_profile['most_common_lat'], user_profile['most_common_lon'])

    new_route_segments = []
    current_location = user_location
    remaining_distance = max_distance

    while remaining_distance > 0:
        remaining_segments = routes_df[routes_df['distance'] <= remaining_distance]
        if remaining_segments.empty:
            break
        next_segment = find_next_segment(current_location, remaining_segments)
        new_route_segments.append(next_segment)
        current_location = (next_segment['latitude'][-1], next_segment['longitude'][-1])
        remaining_distance -= next_segment['distance']

    return pd.concat(new_route_segments, ignore_index=True)

# Test de functie voor een specifieke gebruiker
sample_user_id = 367324
new_route = generate_new_route(sample_user_id)

# Visualiseer de nieuwe route
map_obj = folium.Map(location=[user_profiles_df.loc[user_profiles_df['userId'] == sample_user_id, 'most_common_lat'].iloc[0], user_profiles_df.loc[user_profiles_df['userId'] == sample_user_id, 'most_common_lon'].iloc[0]], zoom_start=12)

for _, segment in new_route.iterrows():
    segment_coords = list(zip(segment['latitude'], segment['longitude']))
    folium.PolyLine(segment_coords, color='blue', weight=2.5, opacity=1).add_to(map_obj)

map_obj


ValueError: setting an array element with a sequence.

In [8]:
import pandas as pd
import numpy as np
import folium
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler

# Laad de gebruikersprofielen en routes
user_profiles_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'
routes_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'
user_profiles_df = pd.read_json(user_profiles_path, lines=True)
routes_df = pd.read_json(routes_path, lines=True)

# Preprocessing en normalisatie
scaler = StandardScaler()

# Voeg een nieuwe kolom 'mean_altitude' toe aan routes_df
routes_df['mean_altitude'] = routes_df['altitude'].apply(np.mean)

# Voeg nieuwe kolommen toe aan user_profiles_df met behulp van scaler.transform
user_profile_features = ['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']
user_profiles_df[user_profile_features] = scaler.fit_transform(user_profiles_df[user_profile_features])

# Voeg nieuwe kolommen toe aan routes_df met behulp van scaler.transform
route_features = ['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']
routes_df[route_features] = scaler.fit_transform(routes_df[route_features])

# Bepaal gewichten voor gebruikerskenmerken en sla ze op in een dictionary
gewichten = user_profiles_df[['userId', 'weight_std_altitude_change', 'weight_std_distance', 'weight_std_condition_value']]
gewichten = gewichten.set_index('userId').to_dict(orient='index')

# Functie om de afstand tussen twee punten te berekenen
def calculate_distance(point1, point2):
    return geodesic(point1, point2).kilometers

# Functie om een aansluitend route segment te vinden
def find_next_segment(current_location, remaining_segments, user_profile_weights):
    min_distance = float('inf')
    closest_segment = None
    
    for index, segment in remaining_segments.iterrows():
        segment_start_location = (segment['latitude'].iat[0], segment['longitude'].iat[0])
        distance_to_segment = calculate_distance(current_location, segment_start_location)

        if distance_to_segment < min_distance:
            min_distance = distance_to_segment
            closest_segment = segment
    
    return closest_segment

# Functie voor het genereren van nieuwe routes
def generate_new_route(user_id, user_profiles_df, routes_df, max_distance=10):
    user_profile = user_profiles_df.loc[user_profiles_df['userId'] == user_id].iloc[0]
    user_location = (user_profile['most_common_lat'], user_profile['most_common_lon'])
    
    new_route_segments = []
    current_location = user_location
    remaining_distance = max_distance

    while remaining_distance > 0:
        remaining_segments = routes_df.loc[routes_df['distance'] <= remaining_distance]
        next_segment = find_next_segment(current_location, remaining_segments, gewichten[user_id])

        if next_segment is not None:
            new_route_segments.append(next_segment)
            current_location = (next_segment['latitude'].iat[-1], next_segment['longitude'].iat[-1])
            remaining_distance -= next_segment['distance']
        else:
            break

    return pd.concat(new_route_segments)

# Test de functie voor een specifieke gebruiker
sample_user_id = 367324
new_route = generate_new_route(sample_user_id, user_profiles_df, routes_df)

# Zoek het gebruikersprofiel voor visualisatie
user_profile = user_profiles_df.loc[user_profiles_df['userId'] == sample_user_id].iloc[0]

# Visualiseer de nieuwe route
map_obj = folium.Map(location=[user_profile['most_common_lat'], user_profile['most_common_lon']], zoom_start=12)
popup_text = f"UserID: {sample_user_id}<br>Average Speed: {user_profile['average_speeds']}<br>Average Altitude Change: {user_profile['average_altitude_changes']}<br>Average Distance: {user_profile['average_distances']}<br>Average Condition Value: {user_profile['average_condition_value']}"
folium.Marker(location=[user_profile['most_common_lat'], user_profile['most_common_lon']], popup=popup_text).add_to(map_obj)

for _, segment in new_route.iterrows():
    segment_coords = list(zip(segment['latitude'], segment['longitude']))
    folium.PolyLine(segment_coords, color='blue', weight=2.5, opacity=1).add_to(map_obj)

map_obj


KeyError: 'most_common_lat'

In [5]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic
import folium
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Pad naar de bestanden
user_profiles_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'
routes_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'

# Laad de gebruikersprofielen en routes
user_profiles_df = pd.read_json(user_profiles_path, lines=True)
routes_df = pd.read_json(routes_path, lines=True)

# Preprocessing van de routegegevens
routes_df['mean_altitude'] = routes_df['altitude'].apply(np.mean)
routes_df['mean_latitude'] = routes_df['latitude'].apply(np.mean)
routes_df['mean_longitude'] = routes_df['longitude'].apply(np.mean)
user_profiles_df[['most_common_lat', 'most_common_lon']] = pd.DataFrame(user_profiles_df['average_location'].tolist(), index=user_profiles_df.index)

# Normaliseer de kenmerken met StandardScaler
scaler = StandardScaler()
user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']] = scaler.fit_transform(user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']])
routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']] = scaler.fit_transform(routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']])

# Bepaal gewichten voor gebruikerskenmerken
gewichten = {}
for index, row in user_profiles_df.iterrows():
    user_id = row['userId']
    gewichten[user_id] = {
        'average_altitude_change': row['weight_std_altitude_change'], 
        'average_distance': row['weight_std_distance'], 
        'average_condition_value': row['weight_std_condition_value'],
        'distance_to_user': 1.0}

# Definieer de functie voor het genereren van nieuwe routes
def generate_new_routes(user_id, user_profiles_df, routes_df, num_segments=5, max_distance=10):
    user_data = user_profiles_df[user_profiles_df['userId'] == user_id].iloc[0]
    user_location = (user_data['most_common_lat'], user_data['most_common_lon'])

    # Selecteer segmenten die voldoen aan gebruikersvoorkeuren
    # Dit is een gesimplificeerde benadering; een meer geavanceerde methode zou nodig zijn voor een realistische implementatie
    candidate_segments = routes_df[routes_df['distance'] < max_distance]

    # Genereer een nieuwe route door segmenten willekeurig te combineren
    new_route = candidate_segments.sample(n=num_segments)
    return new_route

# Test de functie voor een specifieke gebruiker
sample_user_id = 367324
new_route = generate_new_routes(sample_user_id, user_profiles_df, routes_df)

# Visualiseer de nieuwe route
map_obj = folium.Map(location=user_profiles_df.loc[user_profiles_df['userId'] == sample_user_id, ['most_common_lat', 'most_common_lon']].values[0], zoom_start=12)

for _, route_data in new_route.iterrows():
    route_coords = list(zip(route_data['latitude'], route_data['longitude']))
    folium.PolyLine(route_coords, color='blue', weight=2.5, opacity=1).add_to(map_obj)

map_obj

In [ ]:
import random
from geopy.distance import great_circle

def calculate_distance(coord1, coord2):
    """Bereken de afstand tussen twee coördinaten."""
    return great_circle(coord1, coord2).kilometers

def select_and_combine_segments(routes, max_distance, max_altitude_change, proximity_threshold=1.0):
    """
    Selecteer en combineer segmenten van routes die voldoen aan bepaalde criteria.

    Args:
    routes (list): Lijst van route-objecten.
    max_distance (float): Maximale afstand voor de nieuwe route.
    max_altitude_change (float): Maximaal hoogteverschil voor de nieuwe route.
    proximity_threshold (float): Drempel voor de nabijheid van segmenten in kilometers.

    Returns:
    dict: Een nieuw gegenereerde route.
    """
    # Probeer segmenten te combineren tot een geschikte route gevonden is
    for _ in range(100):  # Probeer maximaal 100 keer
        # Selecteer willekeurig twee verschillende segmenten
        segment1, segment2 = random.sample(routes, k=2)

        # Eindpunt van segment1 en startpunt van segment2
        end_coord_segment1 = (segment1['latitude'][-1], segment1['longitude'][-1])
        start_coord_segment2 = (segment2['latitude'][0], segment2['longitude'][0])

        # Controleer of de segmenten dicht genoeg bij elkaar zijn
        if calculate_distance(end_coord_segment1, start_coord_segment2) <= proximity_threshold:
            # Combineer de segmenten
            new_route = {
                'latitude': segment1['latitude'] + segment2['latitude'],
                'longitude': segment1['longitude'] + segment2['longitude'],
                'altitude': segment1['altitude'] + segment2['altitude']
            }

            # Bereken de afstand en hoogteverandering van de nieuwe route
            new_route_distance = segment1['distance'] + segment2['distance']
            new_route_altitude_change = segment1['altitude_change'] + segment2['altitude_change']

            # Controleer of de nieuwe route voldoet aan de criteria
            if new_route_distance <= max_distance and new_route_altitude_change <= max_altitude_change:
                return new_route

    # Geen geschikte combinatie gevonden
    return None

# Voorbeeld van het gebruik van de functie
max_distance_example = 15.0  # Maximale afstand van 15 km
max_altitude_change_example = 50.0  # Maximaal hoogteverschil van 50 meter
new_route_advanced = select_and_combine_segments(filtered_routes_example_corrected_v3, max_distance_example, max_altitude_change_example)

new_route_advanced  # Toon de gegenereerde route (indien beschikbaar)


In [3]:
import numpy as np
import pandas as pd
def generate_simulated_route(average_distance, average_altitude_change, num_points=10):
    """
    Genereer een gesimuleerde route die een geschatte afstand en hoogtemeters heeft.
    De route bestaat uit 'num_points' punten (GPS-coördinaten).
    """
    # Startpunt
    lat, lon = 52.0, 5.0  # Centrale punt in Nederland voor simulatie

    # Genereer willekeurige punten rond dit startpunt
    # De afstand tussen de punten en de hoogteverandering wordt gesimuleerd
    points = []
    total_distance = 0
    total_altitude = 0

    for _ in range(num_points - 1):
        # Simuleer een kleine verandering in positie
        delta_lat = np.random.uniform(-0.01, 0.01)
        delta_lon = np.random.uniform(-0.01, 0.01)

        # Update positie
        lat += delta_lat
        lon += delta_lon
        points.append((lat, lon))

        # Update de gesimuleerde afstand en hoogtemeters
        distance = np.sqrt(delta_lat**2 + delta_lon**2) * 111.32  # Ruwe schatting van km per graad
        total_distance += distance
        total_altitude += np.random.uniform(0, average_altitude_change / num_points)

    # Schaal de route om te voldoen aan de gemiddelde afstand en hoogtemeters
    distance_scale = average_distance / total_distance
    altitude_scale = average_altitude_change / total_altitude
    scaled_points = [(lat * distance_scale, lon * distance_scale) for lat, lon in points]

    return scaled_points

# Voorbeeld van hoe de functie gebruikt kan worden
average_distance = 11.27  # gemiddelde afstand in km
average_altitude_change = 42.10  # gemiddelde hoogtemeters in m
num_points = 50  # aantal punten in de route

full_simulated_route = generate_simulated_route(average_distance, average_altitude_change, num_points)

# De gesimuleerde route
print(full_simulated_route)


[(12.753127938589607, 1.2281691243045156), (12.754031538254447, 1.2265667773492073), (12.752973050609052, 1.2246431976742302), (12.754340715897786, 1.2269181143009344), (12.75420253327233, 1.2272368845893447), (12.755826175120411, 1.2287343183167225), (12.754843564954873, 1.2292691948800258), (12.755762616487543, 1.2315195252918567), (12.756131799340052, 1.229167324714228), (12.756633117851216, 1.2306172413627323), (12.75418505643524, 1.2300731031466208), (12.755798893994161, 1.2295392632943873), (12.755857076460892, 1.2280189839800013), (12.756073435214693, 1.226899493263085), (12.758056170380478, 1.2257951408569518), (12.755748852984704, 1.2256264288171999), (12.7552979863473, 1.2260885149953005), (12.75329822475757, 1.2274439459628348), (12.755079453015549, 1.2289286036555798), (12.75706892041733, 1.2308918092942993), (12.755183184603156, 1.2318235465575371), (12.753299551142002, 1.2333581226239223), (12.752407212746753, 1.2357393802523076), (12.751026652072788, 1.2348193778099739),

In [7]:
import itertools
import pandas as pd

route_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'
all_routes = pd.read_json(route_path, lines=True)

def calculate_total_metrics(segments):
    """ Bereken de totale afstand en hoogtemeters voor een combinatie van segmenten. """
    total_distance = sum(segment['distance'] for segment in segments)
    total_altitude_change = sum(segment['altitude_change'] for segment in segments)
    return total_distance, total_altitude_change

def find_combinations(routes, target_distance, target_altitude_change, max_segments=3, distance_tolerance=1.0, altitude_tolerance=10.0):
    """
    Zoek combinaties van route-segmenten die dicht bij de doelafstand en hoogtemeters liggen.
    """
    combinations = []
    for num_segments in range(1, max_segments + 1):
        for segments in itertools.combinations(routes, num_segments):
            total_distance, total_altitude_change = calculate_total_metrics(segments)
            if abs(total_distance - target_distance) <= distance_tolerance and abs(total_altitude_change - target_altitude_change) <= altitude_tolerance:
                combinations.append(segments)
    
    combinations.sort(key=lambda x: (abs(calculate_total_metrics(x)[0] - target_distance), abs(calculate_total_metrics(x)[1] - target_altitude_change)))
    return combinations

# Parameters
target_distance = 11.27  # Gemiddelde afstand in km
target_altitude_change = 42.10  # Gemiddelde hoogtemeters in m

route_dicts = all_routes.to_dict('records')
route_combinations = find_combinations(route_dicts, target_distance, target_altitude_change)

# Zoek geschikte combinaties van segmenten
route_combinations = find_combinations(route_combinations, target_distance, target_altitude_change)

# Toon de details van de top combinatie (als deze bestaat)
top_combination = route_combinations[0] if route_combinations else []
top_combination_metrics = calculate_total_metrics(top_combination) if top_combination else []

top_combination, top_combination_metrics


KeyboardInterrupt: 

In [9]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
import itertools

# Lees de routegegevens in
route_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'  # Pas dit pad aan naar uw bestandslocatie
all_routes = pd.read_json(route_path, lines=True)

# Lees de gebruikersprofielen in
user_profile_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'  # Vervang dit door het werkelijke pad naar uw bestand
user_profiles = pd.read_json(user_profile_path, lines=True)

# Gebruik de gemiddelde locatie van de eerste gebruiker
first_user_location = user_profiles.iloc[0]['average_location']
user_location = (first_user_location[0], first_user_location[1])  # (latitude, longitude)

# Zet de routes om naar een GeoDataFrame
gdf = gpd.GeoDataFrame(all_routes, geometry=gpd.points_from_xy(all_routes.location.apply(lambda x: x[1]), all_routes.location.apply(lambda x: x[0])))

# Filterfunctie voor routes binnen een bepaalde radius
def filter_routes_within_radius(routes, user_location, radius):
    user_point = Point(user_location)
    bounding_box = user_point.buffer(radius).bounds
    potential_routes = routes.cx[bounding_box[0]:bounding_box[2], bounding_box[1]:bounding_box[3]]
    return potential_routes[potential_routes.distance(user_point) <= radius]

# Filter routes binnen de radius
radius = 10000  # Radius in meters
filtered_routes = filter_routes_within_radius(gdf, user_location, radius)

# Functie om geschikte combinaties van routes te vinden
def find_suitable_combinations(routes, target_distance, target_altitude_change, max_segments=3):
    combinations = []
    for num_segments in range(1, max_segments + 1):
        for segments in itertools.combinations(routes.itertuples(), num_segments):
            total_distance = sum(segment.distance for segment in segments)
            total_altitude_change = sum(segment.altitude_change for segment in segments)
            if abs(total_distance - target_distance) <= target_distance * 0.1 and abs(total_altitude_change - target_altitude_change) <= target_altitude_change * 0.1:
                combinations.append(segments)
    return combinations

# Parameters voor doelkenmerken
target_distance = user_profiles.iloc[0]['average_distances']
target_altitude_change = user_profiles.iloc[0]['average_altitude_changes']
#average_altitude_changes
# Zoek naar geschikte combinaties
suitable_combinations = find_suitable_combinations(filtered_routes, target_distance, target_altitude_change)

# Toon de details van de top combinatie (als deze bestaat)
top_combination = suitable_combinations[0] if suitable_combinations else []
top_combination_metrics = [(segment.distance, segment.altitude_change) for segment in top_combination] if top_combination else []

top_combination_metrics


AttributeError: 'DataFrame' object has no attribute 'location'

In [12]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
import itertools
from shapely.geometry import LineString

def create_linestring_from_lat_lon(lat_list, lon_list):
    return LineString(zip(lon_list, lat_list))

# Lees de routegegevens in
route_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'  # Pas dit pad aan naar uw bestandslocatie
all_routes = pd.read_json(route_path, lines=True)

# Lees de gebruikersprofielen in
user_profile_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'  # Vervang dit door het werkelijke pad naar uw bestand
user_profiles = pd.read_json(user_profile_path, lines=True)

# Gebruik de gemiddelde locatie van de eerste gebruiker
first_user_location = user_profiles.iloc[0]['average_location']
user_location = (first_user_location[0], first_user_location[1])  # (latitude, longitude)

# Zet de routes om naar een GeoDataFrame
gdf = gpd.GeoDataFrame(all_routes, geometry=all_routes.apply(lambda x: create_linestring_from_lat_lon(x['latitude'], x['longitude']), axis=1))

# Filterfunctie voor routes binnen een bepaalde radius
def filter_routes_within_radius(routes, user_location, radius):
    user_point = Point(user_location)
    bounding_box = user_point.buffer(radius).bounds
    potential_routes = routes.cx[bounding_box[0]:bounding_box[2], bounding_box[1]:bounding_box[3]]
    return potential_routes[potential_routes.distance(user_point) <= radius]

# Filter routes binnen de radius
radius = 5000  # Radius in meters
filtered_routes = filter_routes_within_radius(gdf, user_location, radius)

# Functie om geschikte combinaties van routes te vinden
def find_suitable_combinations(routes, target_distance, target_altitude_change, max_segments=3):
    combinations = []
    for num_segments in range(1, max_segments + 1):
        for segments in itertools.combinations(routes.itertuples(), num_segments):
            total_distance = sum(segment.distance for segment in segments)
            total_altitude_change = sum(segment.altitude_change for segment in segments)
            if abs(total_distance - target_distance) <= target_distance * 0.1 and abs(total_altitude_change - target_altitude_change) <= target_altitude_change * 0.1:
                combinations.append(segments)
    return combinations

# Parameters voor doelkenmerken
target_distance = user_profiles.iloc[0]['average_distances']
target_altitude_change = user_profiles.iloc[0]['average_altitude_changes']
#average_altitude_changes
# Zoek naar geschikte combinaties
suitable_combinations = find_suitable_combinations(filtered_routes, target_distance, target_altitude_change)

# Toon de details van de top combinatie (als deze bestaat)
top_combination = suitable_combinations[0] if suitable_combinations else []
top_combination_metrics = [(segment.distance, segment.altitude_change) for segment in top_combination] if top_combination else []

top_combination_metrics


c:\Users\Ruben\AppData\Local\Programs\Python\Python310\lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


KeyboardInterrupt: 

In [ ]:
import json
from geopy.distance import geodesic
import itertools

def calculate_route_distance(coords):
    """ Bereken de afstand van een route op basis van coördinaten. """
    total_distance = 0
    for i in range(1, len(coords)):
        total_distance += geodesic(coords[i - 1], coords[i]).kilometers
    return total_distance

def are_points_close(point1, point2, max_distance=0.1):
    """ Controleer of twee punten binnen een bepaalde afstand van elkaar liggen. """
    return geodesic(point1, point2).kilometers <= max_distance

def find_partial_connecting_routes(routes, target_distance=10, distance_tolerance=2, proximity_threshold=0.1):
    """ Vind aansluitende routes of delen van routes die samen een gespecificeerde afstand benaderen. """
    connected_routes = []

    # Alle combinaties van routes overwegen
    for route1, route2 in itertools.permutations(routes, 2):
        for split_index in range(1, len(route2['coords'])):
            # Deel van route2 nemen
            partial_route2 = route2['coords'][:split_index]
            # Controleer of de punten dichtbij elkaar liggen
            if are_points_close(route1['coords'][-1], partial_route2[0], proximity_threshold):
                combined_coords = route1['coords'] + partial_route2[1:]  # Voorkomen van duplicatie van het aansluitpunt
                combined_distance = calculate_route_distance(combined_coords)
                if abs(combined_distance - target_distance) <= distance_tolerance:  # Vergrote tolerantie
                    connected_routes.append({
                        'combined_coords': combined_coords,
                        'combined_distance': combined_distance
                    })

    return connected_routes

# Bestand inlezen en routes extraheren
file_path = 'jouw_bestandspad_hier.json'
extracted_routes = []
with open(file_path, 'r') as file:
    for line in file:
        route = json.loads(line)
        coords = list(zip(route['latitude'], route['longitude']))
        extracted_routes.append({'coords': coords, 'distance': route['distance']})

# Zoeken naar aansluitende routes met een totale afstand van ongeveer 10 km met een vergrote tolerantie
partial_connected_routes = find_partial_connecting_routes(extracted_routes)

# Resultaten weergeven
for route in partial_connected_routes:
    print("Gecombineerde Afstand: {:.2f} km".format(route['combined_distance']))
    print("Coördinaten:", route['combined_coords'])


In [ ]:
import json
from itertools import combinations

# Function to check if two routes partially overlap
def check_partial_overlap(route1, route2, segment_length=10):
    """Check if a segment of route1 overlaps with any segment of route2. 
    Segment length can be adjusted."""
    for i in range(len(route1) - segment_length + 1):
        segment1 = set(route1[i:i + segment_length])
        for j in range(len(route2) - segment_length + 1):
            segment2 = set(route2[j:j + segment_length])
            if segment1 == segment2:
                return True
    return False

# Path to the file containing the routes
file_path = '/mnt/data/100_routes.json'

# Reading the entire file and parsing each line as a JSON object
routes = []
with open(file_path, 'r') as file:
    for line in file:
        routes.append(json.loads(line))

# Extracting latitude and longitude coordinates for each route
route_coordinates = []
for route in routes:
    latitudes = route.get('latitude', [])
    longitudes = route.get('longitude', [])
    route_coordinates.append(list(zip(latitudes, longitudes)))

# Check for partial overlaps between all pairs of routes
overlaps = {}
for i, j in combinations(range(len(route_coordinates)), 2):
    if check_partial_overlap(route_coordinates[i], route_coordinates[j]):
        overlaps[(i, j)] = True

# Displaying the count and the indices of the overlapping route pairs
len(overlaps), overlaps.keys()


In [1]:
import json
import random
import folium
import haversine

# Functie om de afstand tussen twee coördinaten te berekenen
def bereken_afstand(coord1, coord2):
    return haversine.haversine(coord1, coord2, unit=haversine.Unit.KILOMETERS)

# Stap 1: Data Voorbereiding
def laad_data(file_path):
    routes = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse elke regel als een JSON-object
            route = json.loads(line.strip())
            routes.append(route)
    return routes

# Stap 2: Analyse van de Route Structuur
def identificeer_route_segmenten(routes):
    segmenten = []
    for route in routes:
        # Aanname: De structuur van de route bevat 'latitude' en 'longitude' arrays
        latitudes = route.get('latitude', [])
        longitudes = route.get('longitude', [])
        # Verdeel de route in segmenten, hier eenvoudigweg door de route in tweeën te splitsen
        mid_index = len(latitudes) // 2
        segmenten.append((latitudes[:mid_index], longitudes[:mid_index]))
        segmenten.append((latitudes[mid_index:], longitudes[mid_index:]))
    return segmenten

# Stap 3: Route Generatie
def genereer_nieuwe_routes(segmenten, aantal_routes=10, max_afstand=10):
    nieuwe_routes = []
    for _ in range(aantal_routes):
        nieuwe_route = {'latitude': [], 'longitude': []}

        # Kies een willekeurig startsegment
        start_segment = random.choice(segmenten)
        nieuwe_route['latitude'] = start_segment[0]
        nieuwe_route['longitude'] = start_segment[1]

        # Voeg andere segmenten toe die dichtbij zijn
        for __ in range(random.randint(1, 4)):  # Aantal extra segmenten
            laatste_punt = (nieuwe_route['latitude'][-1], nieuwe_route['longitude'][-1])
            dichtbij_segmenten = [(seg, bereken_afstand(laatste_punt, (seg[0][0], seg[1][0]))) for seg in segmenten]
            dichtbij_segmenten = [seg for seg, dist in dichtbij_segmenten if dist < max_afstand]
            
            if dichtbij_segmenten:
                volgende_segment = random.choice(dichtbij_segmenten)
                nieuwe_route['latitude'] += volgende_segment[0]
                nieuwe_route['longitude'] += volgende_segment[1]

        nieuwe_routes.append(nieuwe_route)
    return nieuwe_routes
# Stap 4: Validatie en Verfijning (vereenvoudigde weergave)
def valideer_routes(routes):
    # Deze functie zou meer complexe logica moeten bevatten om routes te valideren
    # Voor dit voorbeeld doen we een eenvoudige check
    return [route for route in routes if len(route['latitude']) > 0 and len(route['longitude']) > 0]

# Functie om een route te visualiseren op een kaart met folium
def visualiseer_meerdere_routes(routes):
    # Maak een nieuwe kaart aan, gecentreerd op het eerste punt van de eerste route
    if routes:
        m = folium.Map(location=[routes[0]['latitude'][0], routes[0]['longitude'][0]], zoom_start=12)
    else:
        # Als er geen routes zijn, maak een standaardkaart
        m = folium.Map(location=[0, 0], zoom_start=2)

    # Voeg elke route toe als een lijn op de kaart
    for route in routes:
        folium.PolyLine(list(zip(route['latitude'], route['longitude'])), color=random_color(), weight=2.5, opacity=1).add_to(m)
    return m

# Functie om een willekeurige kleur te genereren voor de visualisatie
def random_color():
    import random
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Voer de stappen uit met een voorbeeldbestand
file_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'  # Verander dit naar het werkelijke pad van uw bestand
routes = laad_data(file_path)
segmenten = identificeer_route_segmenten(routes)
nieuwe_routes = genereer_nieuwe_routes(segmenten, 10)
gevalideerde_routes = valideer_routes(nieuwe_routes)

# Visualiseer alle gegenereerde en gevalideerde routes
kaart = visualiseer_meerdere_routes(gevalideerde_routes)
kaart.save('../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html')  # Sla de kaart op als een HTML-bestand

# De code retourneert het pad naar het opgeslagen HTML-bestand
'../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html'


'../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html'

In [9]:
import json
import random
import folium
import haversine

# Functie om de afstand tussen twee coördinaten te berekenen
def bereken_afstand(coord1, coord2):
    return haversine.haversine(coord1, coord2, unit=haversine.Unit.KILOMETERS)

def bereken_route_afstand(route):
    totale_afstand = 0
    for i in range(1, len(route['latitude'])):
        punt1 = (route['latitude'][i - 1], route['longitude'][i - 1])
        punt2 = (route['latitude'][i], route['longitude'][i])
        totale_afstand += bereken_afstand(punt1, punt2)
    return totale_afstand

# Stap 1: Data Voorbereiding
def laad_data(file_path):
    routes = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse elke regel als een JSON-object
            route = json.loads(line.strip())
            routes.append(route)
    return routes

# Stap 2: Analyse van de Route Structuur (aangepast)
def identificeer_route_segmenten(routes):
    segmenten = []
    for route_idx, route in enumerate(routes):
        latitudes = route.get('latitude', [])
        longitudes = route.get('longitude', [])
        mid_index = len(latitudes) // 2
        # Voeg route_idx toe aan elk segment
        segmenten.append(((latitudes[:mid_index], longitudes[:mid_index]), route_idx))
        segmenten.append(((latitudes[mid_index:], longitudes[mid_index:]), route_idx))
    return segmenten

# Stap 3: Route Generatie (aangepast)
def genereer_nieuwe_routes(segmenten, aantal_routes=10, max_afstand=1):
    nieuwe_routes = []
    for _ in range(aantal_routes):
        nieuwe_route = {'latitude': [], 'longitude': [], 'oorspronkelijke_route_ids': set()}
        gebruikte_route_ids = set()

        start_segment, route_id = random.choice(segmenten)
        nieuwe_route['latitude'] = start_segment[0]
        nieuwe_route['longitude'] = start_segment[1]
        nieuwe_route['oorspronkelijke_route_ids'].add(route_id)
        gebruikte_route_ids.add(route_id)

        for __ in range(random.randint(1, 4)):
            laatste_punt = (nieuwe_route['latitude'][-1], nieuwe_route['longitude'][-1])
            dichtbij_segmenten = [
                (seg, bereken_afstand(laatste_punt, (seg[0][0], seg[0][1])), rid) 
                for seg, rid in segmenten if rid not in gebruikte_route_ids
            ]
            dichtbij_segmenten = [seg for seg in dichtbij_segmenten if seg[1] < max_afstand]
            
            if dichtbij_segmenten:
                volgende_segment, _, rid = random.choice(dichtbij_segmenten)
                nieuwe_route['latitude'] += volgende_segment[0]
                nieuwe_route['longitude'] += volgende_segment[1]
                nieuwe_route['oorspronkelijke_route_ids'].add(rid)
                gebruikte_route_ids.add(rid)

        nieuwe_route['oorspronkelijke_route_ids'] = list(nieuwe_route['oorspronkelijke_route_ids'])
        nieuwe_routes.append(nieuwe_route)
    return nieuwe_routes

# Stap 4: Validatie en Verfijning (vereenvoudigde weergave)
def valideer_routes(routes):
    # Deze functie zou meer complexe logica moeten bevatten om routes te valideren
    # Voor dit voorbeeld doen we een eenvoudige check
    return [route for route in routes if len(route['latitude']) > 0 and len(route['longitude']) > 0]

# Functie om een route te visualiseren op een kaart met folium
def visualiseer_meerdere_routes(routes):
    if routes:
        m = folium.Map(location=[routes[0]['latitude'][0], routes[0]['longitude'][0]], zoom_start=12)
    else:
        m = folium.Map(location=[0, 0], zoom_start=2)

    for route in routes:
        afstand = bereken_route_afstand(route)
        oorsprong_info = ', '.join(map(str, route['oorspronkelijke_route_ids']))
        route_info = f"Afstand: {afstand:.2f} km, Oorsprong: {oorsprong_info}"
        folium.PolyLine(
            list(zip(route['latitude'], route['longitude'])), 
            color=random.choice(['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#00FFFF', '#FF00FF']),  # Willekeurige kleur voor elke route
            weight=2.5, 
            opacity=1,
            popup=route_info
        ).add_to(m)
    return m


# Voer de stappen uit met een voorbeeldbestand
file_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'  # Verander dit naar het werkelijke pad van uw bestand
routes = laad_data(file_path)
segmenten = identificeer_route_segmenten(routes)
nieuwe_routes = genereer_nieuwe_routes(segmenten, 1)
gevalideerde_routes = valideer_routes(nieuwe_routes)

# Visualiseer alle gegenereerde en gevalideerde routes
kaart = visualiseer_meerdere_routes(gevalideerde_routes)
kaart.save('../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html')  # Sla de kaart op als een HTML-bestand

# De code retourneert het pad naar het opgeslagen HTML-bestand
'../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html'


'../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html'

In [13]:
import json
import random
import folium
import haversine

# Functie om de afstand tussen twee coördinaten te berekenen
def bereken_afstand(coord1, coord2):
    return haversine.haversine(coord1, coord2, unit=haversine.Unit.KILOMETERS)

def bereken_route_afstand(route):
    totale_afstand = 0
    for i in range(1, len(route['latitude'])):
        punt1 = (route['latitude'][i - 1], route['longitude'][i - 1])
        punt2 = (route['latitude'][i], route['longitude'][i])
        totale_afstand += bereken_afstand(punt1, punt2)
    return totale_afstand

# Stap 1: Data Voorbereiding
def laad_data(file_path):
    routes = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse elke regel als een JSON-object
            route = json.loads(line.strip())
            routes.append(route)
    return routes

# Stap 2: Analyse van de Route Structuur (aangepast)
def identificeer_route_segmenten(routes):
    segmenten = []
    for route_idx, route in enumerate(routes):
        latitudes = route.get('latitude', [])
        longitudes = route.get('longitude', [])
        mid_index = len(latitudes) // 2
        # Voeg route_idx toe aan elk segment
        segmenten.append(((latitudes[:mid_index], longitudes[:mid_index]), route_idx))
        segmenten.append(((latitudes[mid_index:], longitudes[mid_index:]), route_idx))
    return segmenten

# Stap 3: Route Generatie (aangepast)
def genereer_nieuwe_routes(segmenten, aantal_routes=10, max_afstand=5):
    nieuwe_routes = []
    for _ in range(aantal_routes):
        nieuwe_route = {'latitude': [], 'longitude': [], 'oorspronkelijke_route_ids': set()}
        laatste_toegevoegde_route_id = None  # Houdt de ID van de laatst toegevoegde route bij

        start_segment, route_id = random.choice(segmenten)
        nieuwe_route['latitude'] = start_segment[0]
        nieuwe_route['longitude'] = start_segment[1]
        nieuwe_route['oorspronkelijke_route_ids'].add(route_id)
        laatste_toegevoegde_route_id = route_id

        for __ in range(random.randint(1, 4)):
            laatste_punt = (nieuwe_route['latitude'][-1], nieuwe_route['longitude'][-1])
            dichtbij_segmenten = [
                (seg, bereken_afstand(laatste_punt, (seg[0][0], seg[0][1])), rid) 
                for seg, rid in segmenten if rid != laatste_toegevoegde_route_id
            ]
            dichtbij_segmenten = [seg for seg in dichtbij_segmenten if seg[1] < max_afstand]
            
            if dichtbij_segmenten:
                volgende_segment, _, rid = random.choice(dichtbij_segmenten)
                nieuwe_route['latitude'] += volgende_segment[0]
                nieuwe_route['longitude'] += volgende_segment[1]
                nieuwe_route['oorspronkelijke_route_ids'].add(rid)
                laatste_toegevoegde_route_id = rid  # Update de laatste toegevoegde route ID

        nieuwe_route['oorspronkelijke_route_ids'] = list(nieuwe_route['oorspronkelijke_route_ids'])
        nieuwe_routes.append(nieuwe_route)
    return nieuwe_routes


# Stap 4: Validatie en Verfijning (vereenvoudigde weergave)
def valideer_routes(routes):
    # Deze functie zou meer complexe logica moeten bevatten om routes te valideren
    # Voor dit voorbeeld doen we een eenvoudige check
    return [route for route in routes if len(route['latitude']) > 0 and len(route['longitude']) > 0]

# Functie om een route te visualiseren op een kaart met folium
# Functie om een route te visualiseren op een kaart met folium
def visualiseer_meerdere_routes(routes):
    if routes:
        m = folium.Map(location=[routes[0]['latitude'][0], routes[0]['longitude'][0]], zoom_start=12)
    else:
        m = folium.Map(location=[0, 0], zoom_start=2)

    for route in routes:
        afstand = bereken_route_afstand(route)
        oorsprong_info = ', '.join(map(str, route['oorspronkelijke_route_ids']))
        route_info = f"Afstand: {afstand:.2f} km, Oorsprong: {oorsprong_info}"
        folium.PolyLine(
            list(zip(route['latitude'], route['longitude'])), 
            color=random.choice(['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#00FFFF', '#FF00FF']),  # Willekeurige kleur voor elke route
            weight=2.5, 
            opacity=1,
            popup=route_info
        ).add_to(m)
    return m  # Retourneer het Folium-kaartobject

# Voer de stappen uit en visualiseer de kaart direct
file_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'  # Verander dit naar het werkelijke pad van uw bestand
routes = laad_data(file_path)
segmenten = identificeer_route_segmenten(routes)
nieuwe_routes = genereer_nieuwe_routes(segmenten, 1)
gevalideerde_routes = valideer_routes(nieuwe_routes)

# Visualiseer alle gegenereerde en gevalideerde routes
kaart = visualiseer_meerdere_routes(gevalideerde_routes)
kaart  # Dit toont de kaart direct in een Jupyter Notebook of vergelijkbare omgeving

